PROCESO A ALTO NIVEL



PROCESO DE ESTE EJEMPLO

objetivo: clasifdicar cifar-10

como modelo pre-entrenado usaremos vgg-16

; la desc. de vgg-16 en el anexo para no mezclar con el flujo principal



In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

2025-09-25 19:18:23.375546: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data

In [2]:
# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# nuetsro modelo base será vgg-16 que necesita imgs. 224x224 y cifra 10 son..

In [3]:
# Preprocess input for VGG16 (scales pixel values in the way VGG16 expects)
x_train_preprocessed = preprocess_input(x_train)
x_test_preprocessed = preprocess_input(x_test)

# One-hot encode labels
y_train_cat = to_categorical(y_train, 10) # dsp- sin hardcoding
y_test_cat = to_categorical(y_test, 10)

In [4]:
x_train_preprocessed.shape

(50000, 32, 32, 3)

In [5]:
y_train_cat[0] # is one hot encoded

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

## Model

In [17]:
# dibujo

# Feature extraction using VGG16
def feature_extractor(inputs):
    vgg = tf.keras.applications.VGG16(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    vgg.trainable = False  # Freeze feature extractor
    return vgg(inputs)

# Classifier head
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs) # add another dense later
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

# Final model combining resize, feature extractor, and classifier
def final_model(inputs):
    resize = tf.keras.layers.Resizing(height=224, width=224)(inputs)  # resize; from 32x32 to 224x224
    vgg_features = feature_extractor(resize)
    classification_output = classifier(vgg_features)
    return classification_output

# Define and compile the model
def define_compile_model():
    inputs = tf.keras.layers.Input(shape=(32, 32, 3))
    output = final_model(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=output)

    model.compile(
        optimizer='SGD',
        loss='categorical_crossentropy',  # labels are one-hot encoded
        metrics=['accuracy']
    )
    return model

# Instantiate the model
model = define_compile_model()
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing_1 (Resizing)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification (Dense)          │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,719,818 (56.15 MB)

 Trainable params: 5,130 (20.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
# Train
print("Stage 1: Training classifier only (feature extraction)")
model.fit(x_train_preprocessed, y_train_cat,
          validation_data=(x_test_preprocessed, y_test_cat),
          epochs=2,
          batch_size=64)

Stage 1: Training classifier only (feature extraction)
Epoch 1/2


2025-09-25 19:16:34.724583: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 614400000 exceeds 10% of free system memory.
2025-09-25 19:16:35.720784: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 1644167168 exceeds 10% of free system memory.


## Fine tuning

Step 2: Unfreeze last VGG16 block for fine-tuning

We need to:

Access the VGG16 model inside feature_extractor.

Unfreeze only the last block (block5) layers.

Recompile the model with a smaller learning rate.

Here’s a practical way:

In [18]:
# Access the VGG16 layer in the model
vgg_layer = None
for layer in model.layers:
    if isinstance(layer, tf.keras.Model) and 'vgg16' in layer.name:
        vgg_layer = layer
        break

if vgg_layer is None:
    print("VGG16 layer not found!")
else:
    # Unfreeze last conv block (block5)
    vgg_layer.trainable = True
    for layer in vgg_layer.layers:
        if not layer.name.startswith('block5'):
            layer.trainable = False



In [19]:
vgg = model.get_layer('vgg16')  # or whatever name you used
for layer in vgg.layers:
    print(layer.name, layer.trainable)

input_layer_6 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [21]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing_1 (Resizing)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification (Dense)          │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,719,818 (56.15 MB)

 Trainable params: 7,084,554 (27.03 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [ ]:
# it has more trainable params.

Calling model.compile() again does NOT reset or forget the learned weights.

When you call model.fit(), training updates the model's weights.

Changing layer.trainable flags changes which weights will be updated in subsequent training.

Calling model.compile() again only updates the training configuration — e.g., optimizer, loss, metrics, learning rate.

The model’s weights stay intact across recompiles, so previously learned information is preserved.

So your workflow is correct for fine-tuning:

Initially, you train the model (usually with some layers frozen).

Then you unfreeze some layers (e.g., 'block5' layers).

You recompile the model with a lower learning rate optimizer.

Finally, you call fit() again to continue training those unfrozen layers.

This will fine-tune those layers without losing the previous training progress.

What does compile do?

Compile defines the loss function, the optimizer and the metrics. That's all.

It has nothing to do with the weights and you can compile a model as many times as you want without causing any problem to pretrained weights.

In [ ]:
# Recompile with a smaller learning rate
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9),
    loss='categorical_crossentropy',  # still using one-hot labels
    metrics=['accuracy']
)

print("Stage 2: Fine-tuning last VGG block")
model.fit(
    x_train_preprocessed, y_train_cat,
    validation_data=(x_test_preprocessed, y_test_cat),
    epochs=5,  # can increase
    batch_size=64
)

## _Annex

In [8]:
vgg = tf.keras.applications.VGG16(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
vgg.trainable = False  # Freeze feature extractor

In [9]:
vgg.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [10]:
# output is (batch_size, 7, 7, 512)

Output shape calculation

Input: (224, 224, 3)

VGG16 has 5 convolutional blocks with MaxPooling2D(pool_size=2) after each block.

So the spatial dimensions halve after each pooling layer:

```
| Block | Input size | After pooling |
| ----- | ---------- | ------------- |
| 1     | 224×224    | 112×112       |
| 2     | 112×112    | 56×56         |
| 3     | 56×56      | 28×28         |
| 4     | 28×28      | 14×14         |
| 5     | 14×14      | 7×7           |
```

Number of channels after last conv block = 512

In [11]:
for layer in model.layers:
    print(layer.name, layer.trainable)


input_layer_2 True
resizing True
vgg16 False
global_average_pooling2d_1 True
classification True


In [13]:
vgg = model.get_layer('vgg16')  # or whatever name you used
for layer in vgg.layers:
    print(layer.name, layer.trainable)


input_layer_3 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


To unfreeze

In [14]:
# Assume vgg is your VGG16 model
vgg.trainable = True  # Make all layers trainable first

# Freeze all layers except the last block
for layer in vgg.layers:
    if not layer.name.startswith('block5'):
        layer.trainable = False


In [15]:
vgg = model.get_layer('vgg16')  # or whatever name you used
for layer in vgg.layers:
    print(layer.name, layer.trainable)

    # dsp. re-compile with a lower lr

input_layer_3 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True
